In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [2]:
# restore all of our data structures
import pickle
data = pickle.load( open( "chatbot_invoicing_training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_chatbot_invoicing_logs')

In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [5]:
p = bow("i want to submit payment", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0
 0]
[u'goodbye', u'greeting', u'instructions', u'payment', u'paymentregistration', u'paymentvalidate', u'startpayment', u'thanks']


In [6]:
# load our saved model
model.load('./chatbot_invoicing_model.tflearn')

INFO:tensorflow:Restoring parameters from /notebooks/chatbot_invoicing_model.tflearn


In [49]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        print(random.choice(i['responses']))
                    return

            results.pop(0)

In [77]:
response('Hi')

Hello, thanks for asking


In [78]:
response('I would like to create new payment, what should I do?')

Payment must be managed in Payment Maintenance module


In [64]:
response('Ok, and what about payment validation?', show_details=True)

('tag:', u'paymentvalidate')
Call finance support desk 187073663829


In [65]:
response('Ok, Im ready to pay, what should I do', show_details=True)

('tag:', u'paymentregistration')
Payment must be managed in Payment Maintenance module


In [66]:
response('Ok, I would like to submit payment now', show_details=True)

('context:', u'transaction')
('tag:', u'startpayment')
Do you want to issue payment or check instructions?


In [67]:
response('payment', show_details=True)

('tag:', u'payment')
To execute payment use code VO-10002-8289


In [68]:
context

{'123': u'transaction'}

In [69]:
response('Hi again', show_details=True)

('context:', u'')
('tag:', u'greeting')
Hello, thanks for asking


In [70]:
response('payment', show_details=True)